In [1]:
%matplotlib qt
import serial
import time
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from collections import deque
import re
import numpy as np

In [2]:
PORT = 'COM7'  # Replace with your correct COM port
BAUD = 115200

try:
    ser = serial.Serial(PORT, BAUD)
    time.sleep(2)  # Wait for Arduino to reset
    print(f"Connected to {PORT} at {BAUD} baud.")
except Exception as e:
    print(f"Failed to connect: {e}")


Connected to COM7 at 115200 baud.


In [3]:
max_len = 200
angles = deque([0]*max_len, maxlen=max_len)
times = deque([0]*max_len, maxlen=max_len)
start_time = time.time()

fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)
ax.set_ylim(-180, 180)
ax.set_xlim(0, 10)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Angle (°)')
ax.set_title('Live Rotary Angle')
ax.grid(True)

In [4]:
def update(frame):
    try:
        raw_line = ser.readline().decode('utf-8').strip()
        # print(f"RAW: {raw_line}")
        match = re.search(r"Angle:\s*(-?\d+\.?\d*)", raw_line)

        if match:
            angle = float(match.group(1))
            t = time.time() - start_time
            angles.append(angle)
            times.append(t)

            line.set_data(times, angles)

            # Scroll x-axis to show last 10 seconds
            xmin = max(0, t - 10)
            xmax = t + 1
            ax.set_xlim(xmin, xmax)

            # Optional: custom tick marks every 2 seconds
            ax.set_xticks(np.arange(xmin, xmax, 2))
            

    except Exception as e:
        print(f"Error: {e}")

    return line,


In [ ]:
ani = animation.FuncAnimation(fig, update, interval=30, blit=False, cache_frame_data=False)
plt.tight_layout()
plt.show()

Error: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))
